In [ ]:
# Lake Administrator Demo Notebook
## List of activities
## 0 - Get Lake Formation Administrator access - TODO - Use Boto3 to get this access. 
## 1 - Set Account level lake formation settings for CreateDatabaseDefaultPermissions and CreateTableDefaultPermissions
## 2 - Delete Lakeformation Policy tags
## 3 - Create Lakeformation Policy tags


In [ ]:
# Imports
import json 
import boto3
import logging
import pprint
from pandas import DataFrame
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()

In [ ]:
# Import orbit helpers
from lakeformation_utils import *
from aws_orbit_sdk.database import get_athena
from aws_orbit_sdk.common import get_workspace,get_scratch_database
workspace = get_workspace()
catalog_id = workspace['EksPodRoleArn'].split(':')[-2] 

orbit_data_lake_admin_role_arn = workspace['EksPodRoleArn']
orbit_data_lake_creartor_role_arn = workspace['EksPodRoleArn'].replace("-admin-", "-creator-")
orbit_data_lake_user_role_arn = workspace['EksPodRoleArn'].replace("-admin-", "-user-")

catalog_id = orbit_data_lake_admin_role_arn.split(':')[-2] 

team_space = workspace['team_space']
assert team_space == 'lake-admin'


In [ ]:
# Clients
lfc = boto3.client('lakeformation')
iamc = boto3.client('iam')
ssmc = boto3.client('ssm')
gluec = boto3.client('glue')

In [ ]:
# Step 0 - Get account lake formation settings

In [ ]:
lf_get_account_setting_response = lfc.get_data_lake_settings(
    CatalogId=catalog_id
)

In [ ]:
assert 200 == lf_get_account_setting_response['ResponseMetadata']['HTTPStatusCode']

In [ ]:
pprint.pprint(lf_get_account_setting_response)

# Step 1  
## Add orbit lake admin to account lakeformation administrators
## Change account lake formation default permissions for newly created databases and tables

In [ ]:
if 200 == lf_get_account_setting_response['ResponseMetadata']['HTTPStatusCode']:
    data_lake_principal_identifiers = lf_get_account_setting_response['DataLakeSettings']['DataLakeAdmins']
    account_lf_admin_arns = [lake_admin['DataLakePrincipalIdentifier'] for lake_admin in data_lake_principal_identifiers]


In [ ]:
data_lake_principal_identifiers

In [ ]:
if orbit_data_lake_admin_role_arn not in account_lf_admin_arns:
    data_lake_principal_identifiers.append({'DataLakePrincipalIdentifier': orbit_data_lake_admin_role_arn})

In [ ]:
data_lake_principal_identifiers

In [ ]:
lf_put_account_setting_response = lfc.put_data_lake_settings(
    CatalogId=catalog_id,
    DataLakeSettings={
        'DataLakeAdmins': data_lake_principal_identifiers,
        'CreateDatabaseDefaultPermissions': [],
        'CreateTableDefaultPermissions': []
    }
)


In [ ]:
assert 200 == lf_put_account_setting_response['ResponseMetadata']['HTTPStatusCode']

# Step 2 - Create Lakeformation Policy tags for the demo

In [ ]:
try:
    delete_lf_tag_response = lfc.delete_lf_tag(
        CatalogId= catalog_id,
        TagKey='security-level'    
    )
except Exception as e:
    print('Creating lake formation policy tags for the first time.')
    print(e)

In [ ]:
create_lf_tag_response = lfc.create_lf_tag(
    CatalogId= catalog_id,
    TagKey='security-level',
    TagValues=[
        'sec-1',
        'sec-2',
        'sec-3',
        'sec-4',
        'sec-5',
    ]
)

In [ ]:
assert 200 == create_lf_tag_response['ResponseMetadata']['HTTPStatusCode']

# Step 3 - Add policy tag permissions to orbit lake-creator IAM role.

## Adding DESCRIBE and ASSOCIATE permissions with grant option too.


In [ ]:
# Revisit if we need with grant option

In [ ]:
try:
    lake_creator_revoke_permissions_response = lfc.revoke_permissions(
        CatalogId=catalog_id,
        Principal={
            'DataLakePrincipalIdentifier': orbit_data_lake_creartor_role_arn
        },
        Resource={
            'LFTag': {
                'CatalogId': catalog_id,
                'TagKey': 'security-level',
                'TagValues': [
                    'sec-1',
                    'sec-2',
                    'sec-3',
                    'sec-4',
                    'sec-5',
                ]
            }
        },
        Permissions=[ 'DESCRIBE', 'ASSOCIATE' ],
        PermissionsWithGrantOption=['DESCRIBE', 'ASSOCIATE' ]
    )
except Exception as e:
    print(f'Granting Lakeformation policy tag permissions to {orbit_data_lake_creartor_role_arn} first time.')    
    print(e)

In [ ]:
lake_creator_grant_permissions_response = lfc.grant_permissions(
    CatalogId=catalog_id,
    Principal={
        'DataLakePrincipalIdentifier': orbit_data_lake_creartor_role_arn
    },
    Resource={
        'LFTag': {
            'CatalogId': catalog_id,
            'TagKey': 'security-level',
            'TagValues': [
                'sec-1',
                'sec-2',
                'sec-3',
                'sec-4',
                'sec-5',
            ]
        }
    },
    Permissions=[ 'DESCRIBE', 'ASSOCIATE' ],
    PermissionsWithGrantOption=['DESCRIBE', 'ASSOCIATE' ]
)

In [ ]:
assert 200 == lake_creator_grant_permissions_response['ResponseMetadata']['HTTPStatusCode']

# Step 4 - Add DESCRIBE and SELECT permissions for Lakeformation policy tag security-level:sec-5 to orbit lake user IAM role.

In [ ]:
lu_db_grant_permissions_response = lfc.grant_permissions(
    CatalogId=catalog_id,
    Principal={
        'DataLakePrincipalIdentifier': orbit_data_lake_user_role_arn
    },
    Resource={
        'LFTagPolicy': {
                    'CatalogId': catalog_id,
                    'ResourceType': 'DATABASE',
                    'Expression': [
                        {
                            'TagKey': 'security-level',
                            'TagValues': [
                                'sec-5',
                            ]
                        },
                    ]
                }
    },
    Permissions=[
        'DESCRIBE'
    ],
    
)

In [ ]:
assert 200 == lu_db_grant_permissions_response['ResponseMetadata']['HTTPStatusCode']

In [ ]:
lu_db_table_grant_permissions_response = lfc.grant_permissions(
    CatalogId=catalog_id,
    Principal={
        'DataLakePrincipalIdentifier': orbit_data_lake_user_role_arn
    },
    Resource={
        'LFTagPolicy': {
                    'CatalogId': catalog_id,
                    'ResourceType': 'TABLE',
                    'Expression': [
                        {
                            'TagKey': 'security-level',
                            'TagValues': [
                                'sec-5',
                            ]
                        },
                    ]
                }
    },
    Permissions=[
        'SELECT'
    ],
    
)

In [ ]:
assert 200 == lu_db_table_grant_permissions_response['ResponseMetadata']['HTTPStatusCode']

# End of Orbit lake admin demo notebook.